In [ ]:
from data_gen import *
from models import *

In [ ]:
# data generator
dg = DataGen(data="data/wolaytta-train.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


In [ ]:
14780 * .7

10346.0

In [4]:
14780 * .7

Total train data:  561794.1


In [ ]:
print("Total train data: ", len(dg.words) * .1)
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) * .7 / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.cnn_gen_data_multi_word(batch_size=batch_size, n_batches=n_batches)

In [5]:
# infenc - inference encoder model
# infdec - inference decoder model
# train - training model that combines both
# n_input_length - the length of the input and the output
# word_feat_len - the length of the word feature vector
# n_units - size of the hidden memory in the RNN
train, infenc, infdec = conv_multi_model(n_input_length, n_input_length, dg.word_feat_len + 3, 256)
train.compile(optimizer='adam', loss='categorical_crossentropy')


In [6]:
# model train given the data generator, how many batches and number of epochs
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 3)

W0721 00:41:20.982088 139671214438208 deprecation.py:323] From /home/leo/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4
4389/4389 [==============================] - 161s 37ms/step - loss: 0.1676
Epoch 2/4
4389/4389 [==============================] - 178s 41ms/step - loss: 0.0121
Epoch 3/4
4389/4389 [==============================] - 166s 38ms/step - loss: 0.0056
Epoch 4/4
4389/4389 [==============================] - 156s 35ms/step - loss: 0.0035


In [7]:
dg2 = DataGen(data="data/wolaytta-test.txt")
test_n_batches, test_batch_size =  int(len(dg2.words) * 1. / batch_size), batch_size  
# test_n_batches, test_batch_size = 30, 10 

# data generator for test data
test_gen = dg2.cnn_gen_data_multi_word(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

6


In [8]:
def predict(infenc, infdec, inputs, n_steps, cardinality):
    # encode
    state = infenc.predict(inputs)
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h= infdec.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [9]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
#         word_index = X4[j].reshape((1, X4.shape[1]))
        # predicts the target word given root word and features
        target = predict(infenc, infdec, [root_word_matrix, word_features], n_steps_out, n_input_length)
#         root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
#         word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
#         targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal
        if ''.join(dg.one_hot_decode(y[j])).strip() == ''.join(dg.one_hot_decode(target)).strip():
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 61.74%


In [10]:
# lines = open("data/wol-multi.txt").readlines()
for i in range(10):
    print(dg.roots[i], dg.words[i], dg.word_indexes[i])

tifilatt tifilattaree [0, 1, 0]
xoon xoonissogeeta [0, 0, 1]
qas qasabeakka [1, 0, 0]
baq baqissikkii [1, 0, 0]
onccilatt onccilattaasa [1, 0, 0]
bogg boggssiyogeetu [1, 0, 0]
afuxx afuxxssakka [1, 0, 0]
qayc qaycissiyagee [1, 0, 0]
teeliteel teeliteelissidori [1, 0, 0]
yaat yaatissogeeta [0, 0, 1]


In [11]:
# word2feat, word2root= {}, {}

In [12]:
# for line in lines:
#     line = line[:-1].split(' ')
#     word2feat[line[0]] = line[1]
#     word2root[line[0]] = line[2]

In [13]:
# check = {}
# for word in word2feat:
#     root = word2feat[word]
#     feat = word2root[word]
#     key = root + " " + feat
#     if key not in check:
#         check[key] = []
#     check[key].append(word)
    

In [14]:
# counter = 0
# file = open("data/wol-m.txt", "w")
# for key in check.keys():
#     words = check[key]
#     for i in range(len(words)):
#         word = words[i]
#         line = "{0} {1} {2}\n".format(word, key, i)
#         file.write(line)
# file.close()